In [2]:
#Dependencies
import timeit
from tqdm import tqdm
import pickle
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import missingno
import re
from config import * 
#from harvesine_distance import harvesine_distance
from datetime import datetime

# Extract

## Get urls of the listing for the 3 rental categories

In [673]:
url_longTerm_rental="https://www.kijiji.ca/b-apartments-condos/city-of-toronto"
url_longTerm_base="/c37l1700273"
pageNos = '/page-'
url_shortTerm_rental="https://www.kijiji.ca/b-short-term-rental/city-of-toronto"
url_shortTerm_base="/c42l1700273"
url_roomRoomates_rental="https://www.kijiji.ca/b-room-rental-roommate/city-of-toronto"
url_roomRoomates_base="/c36l1700273"
error_list=[]

In [674]:
def kijiji_getUrls(url, base):
    url_list=[]
    for i in tqdm(range(10)):
        url_final = url+pageNos+str(i)+base
        response = requests.get(url_final)
        soup = bs(response.text, "html.parser")
        results= soup.find_all("div", class_="search-item")
        for r in results:
            url_list.append("https://www.kijiji.ca"+r.a["href"])
    return(url_list)

In [223]:
url_list_longterm=kijiji_getUrls(url_longTerm_rental, url_longTerm_base)


In [224]:
with open('kijiji_longtermRental.pkl', 'wb') as f:
    pickle.dump(url_list_longterm, f)

#with open('kijiji_longtermRental.pkl', 'rb') as f:
    #kijiji_list_longterm = pickle.load(f)

In [225]:
url_list_shortterm=kijiji_getUrls(url_shortTerm_rental, url_shortTerm_base)
with open('kijiji_shorttermRental.pkl', 'wb') as f:
    pickle.dump(url_list_shortterm, f)

#with open('kijiji_shorttermRental.pkl', 'rb') as f:
    #kijiji_list_shorttermRental = pickle.load(f)

url_list_roomRoomates=kijiji_getUrls(url_roomRoomates_rental, url_roomRoomates_base)
with open('kijiji_roomRoomatesRental.pkl', 'wb') as f:
    pickle.dump(url_list_roomRoomates, f)

#with open('kijiji_roomRoomatesRental.pkl', 'rb') as f:
    #kijiji_list_roomRoomatesRental = pickle.load(f)

In [226]:
url="https://www.kijiji.ca/v-apartments-condos/city-of-toronto/renovated-bachelor-for-rent-forest-hill/1508850924"
url2="https://www.kijiji.ca/v-apartments-condos/city-of-toronto/luxury-condo-2-bdrm-2-baths-yonge-st-clair/1516651347"
url3="https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/600-incredible-discount-200-less-per-month-from-august-15/1517738651"
url4= "https://www.kijiji.ca/v-short-term-rental/city-of-toronto/downtown-fully-furnished-yonge-front/1517081941"

## Scrape each listing in each url

In [227]:
def kijiji_url_scrape_LongTerm(kijiji_url):
    
    #driver = webdriver.Chrome()
    #driver.get(kijiji_url)
    response = requests.get(kijiji_url)
    soup = bs(response.text, "html.parser")
    time.sleep(1)
    #page_source = driver.page_source
    #soup=bs(page_source, "html.parser")
    
        ad_id= soup.find("a", class_="adId-4111206830").text.strip()
    
        images=[]
        img_result=soup.find_all("img", class_="heroImageForPrint-1152389693")
        for img in img_result:
            images.append(img["src"])
        title= soup.find("h1", class_="title-2323565163").text.strip()
        address= soup.find("span", class_="address-3617944557").text.strip()
        price= soup.find("div", class_="priceWrapper-1165431705").span.text.strip()
        post_published_date= soup.find("div", class_="datePosted-383942873").time["datetime"]
        rental_type= soup.find_all("span", class_="noLabelValue-3861810455")[0].text.strip()
        bedroom_no= soup.find_all("span", class_="noLabelValue-3861810455")[1].text.strip()
        bathroom_no= soup.find_all("span", class_="noLabelValue-3861810455")[2].text.strip()
        list_results= soup.find_all("ul", class_="list-1757374920")[1]

        utilities_list=[]
        utilities= list_results.find_all("svg")
        for util in utilities:
            utilities_list.append(util["aria-label"])
        if(len(utilities_list)==0):
            utilities_list.append("Not Included")
        wifi_and_more=soup.find_all("ul", class_="list-1757374920")[2].text.strip()


        appliances=[]
        results= soup.find_all("ul", class_="list-1757374920")[4]
        for r in results:
            if(r.text):
                appliances.append(r.text.strip())

        personal_outdoor_space= soup.find_all("ul", class_="list-1757374920")[5].text.strip()

        amenities= []
        results= soup.find_all("ul", class_="list-1757374920")[7]
        for r in results:
            if(r.text):
                amenities.append(r.text.strip())
        description= soup.find("div", class_="descriptionContainer-3544745383").p.text.strip()

        dict_attributes={
            'ad_id': ad_id,
            'images': images,
            'title': title,
            'address':address,
            'price': price,
            'post_published_date': post_published_date,
            'rental_type': rental_type,
            'bedroom_no': bedroom_no,
            'bathroom_no': bathroom_no,
            'utilities_list': utilities_list,
            'wifi_and_more': wifi_and_more,
            'appliances':appliances,
            'personal_outdoor_space': personal_outdoor_space,
            'amenities': amenities,
            'description': description,
            'url': kijiji_url
        }

        dt_list=soup.find_all("dt")
        for i in range(0,len(dt_list)):
            dict_attributes[soup.find_all("dt")[i].text]=soup.find_all("dd")[i].text
    except:
        #error_list.append(kijiji_url)
        return(kijiji_url)
    return(dict_attributes)


IndentationError: unexpected indent (<ipython-input-227-0cbcde97fae7>, line 11)

In [228]:
#kijiji_list_scrape_LongTerm(url2)

In [229]:
def kijiji_url_scrape_other(kijiji_url):
    try:
        #driver = webdriver.Chrome()
        #driver.get(kijiji_url)
        response = requests.get(kijiji_url)
        soup = bs(response.text, "html.parser")
        time.sleep(1)
        #page_source = driver.page_source
        #soup=bs(page_source, "html.parser")

        ad_id= soup.find("a", class_="adId-4111206830").text.strip()
        
        images=[]
        img_result=soup.find_all("img", class_="heroImageForPrint-1152389693")
        for img in img_result:
            images.append(img["src"])
            
        title= soup.find("h1", class_="title-2323565163").text.strip()
        address= soup.find("span", class_="address-3617944557").text.strip()
        price= soup.find("span", class_="currentPrice-2842943473").span.text.strip()
        post_published_date= soup.find("div", class_="datePosted-383942873").time["datetime"]
        rental_type= soup.find("h1", class_="crumbH1-3559413592").span.text.strip()
        description= soup.find("div", class_="descriptionContainer-3544745383").p.text.strip()
        dict_attributes={
            'ad_id': ad_id,
            'images': images,
            'title': title,
            'address':address,
            'price': price,
            'post_published_date': post_published_date,
            'rental_type': rental_type,
            'description': description,
            'url': kijiji_url
        }

        dt_list=soup.find_all("dt")
        for i in range(0,len(dt_list)):
            dict_attributes[soup.find_all("dt")[i].text]=soup.find_all("dd")[i].text
    except:
        #error_list.append(kijiji_url)
        return(kijiji_url)
    return(dict_attributes)


In [230]:
def kijiji_scrape():
    records=[]
    for url in tqdm(kijiji_list_longterm):
        records.append(kijiji_url_scrape_LongTerm(url))
        
    for url in tqdm(kijiji_list_shorttermRental):
        records.append(kijiji_url_scrape_other(url))
        
    for url in tqdm(kijiji_list_roomRoomatesRental):
        records.append(kijiji_url_scrape_other(url))
    return(records)

In [231]:
#records= kijiji_scrape()

# Transform

In [3]:
#with open('kijiji_housing_enriched.pkl', 'wb') as f:
#    pickle.dump(records, f)

with open('kijiji_housing_enriched.pkl', 'rb') as f:
    kijiji_renting_data = pickle.load(f)

In [4]:
kijiji_clean_data=[]
for listing in kijiji_renting_data:
    if str(type(listing))!="<class 'str'>":
        kijiji_clean_data.append(listing)

In [5]:
DF = pd.DataFrame(kijiji_clean_data)
DF.head()

ad_id                                             images  \
0  1515948914  [https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...   
1  1505801433  [https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/jms...   
2  1502301137  [https://i.ebayimg.com/00/s/NjAwWDgwMA==/z/cIk...   
3  1505788420  [https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/NmU...   
4  1516085720  [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/fcY...   

                                               title  \
0  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...   
1  Open House August 12th from 7:00pm to 9:00pm w...   
2  One, two bedroom large unit from $1650 , Hydro...   
3  1 Bedroom Renovated Apartment for Rent - 135 T...   
4  Yonge and Finch, 1+1 unit , spacious and brigh...   

                                          address      price  \
0  100 Parkway Forest Drive, Toronto, ON, M2J 1L6  $2,209.00   
1          70 Spadina  Road, Toronto, ON, M5R 2T6  $1,587.00   
2   40 Tuxedo Ct, Scarborough, ON M1G 3S7, Canada  $1,650.00   
3        135 Tyndall Avenue, Toronto, ON, M6K 2G4  $1,949.00   
4                                 M2N 5R6, Canada  $2,100.00   

        post_published_date rental_type                 bedroom_no  \
0  2020-08-13T22:34:37.000Z   Apartment                Bedrooms: 2   
1  2020-08-13T14:02:28.000Z   Apartment  Bedrooms: Bachelor/Studio   
2  2020-07-17T14:08:19.000Z   Apartment                Bedrooms: 1   
3  2020-08-13T19:06:36.000Z   Apartment                Bedrooms: 1   
4  2020-08-06T02:49:14.000Z       Condo          Bedrooms: 1 + Den   

    bathroom_no                      utilities_list  ... Agreement Type  \
0  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...         1 Year   
1  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...         1 Year   
2  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...         1 Year   
3  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...         1 Year   
4  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...         1 Year   

        Move-In Date Pet Friendly    Size (sqft) Furnished Air Conditioning  \
0    August 13, 2020          Yes            940        No               No   
1  September 1, 2020          Yes  Not Available        No               No   
2      July 24, 2020      Limited            750        No               No   
3    August 13, 2020          Yes            626        No               No   
4     August 4, 2020           No            675        No              Yes   

  Smoking Permitted Barrier-free Entrances and Ramps Visual Aids  \
0               Yes                              NaN         NaN   
1     Outdoors only                              NaN         NaN   
2     Outdoors only                              NaN         NaN   
3                No                              NaN         NaN   
4                No                              Yes          No   

  Accessible Washrooms in Suite  
0                           NaN  
1                           NaN  
2                           NaN  
3                           NaN  
4                           Yes  

[5 rows x 27 columns]

### Remove Duplicates

In [6]:
DF.shape

(737, 27)

In [7]:
DF.drop_duplicates(subset="ad_id", keep="first", inplace=True)

In [8]:
#Check for nan
missingdata_cols = DF.columns[DF.isnull().any()].to_list()
#visualize nan
_=missingno.matrix(DF[missingdata_cols], color=(0.65,0.84,0.84))

In [9]:
DF['geotag']=None

In [10]:
def geocode(addresses):
    #Input: a dictionary with key as index and value as address
    #Output: a dictionary with key as index and value as (lat, long) tuple
    lat_long = {}
    for index in addresses:
        try:
            url = geocode_api+f'?address={addresses[index]}'+f'&key={google_api}'
            url = re.sub(' +', '%20', url)
            response = requests.get(url)
            if response.ok:
                content = response.json()
                lat_long[index]=tuple(content['results'][0]['geometry']['location'].values())
            else:
                lat_long[index]=None
        except Exception as e:
            print(e)
            lat_long[index]=None
    return lat_long

### Geolocation based on address

In [11]:
addresses = DF[DF['geotag'].isnull()]['address'].to_dict()
lat_long = geocode(addresses)
DF['geotag'] = lat_long.values()
#DF.loc[lat_long.keys(), 'geotag'] = DF.loc[lat_long.keys()].index.map(lat_long)

list index out of range


In [12]:
DF.head()

ad_id                                             images  \
0  1515948914  [https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...   
1  1505801433  [https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/jms...   
2  1502301137  [https://i.ebayimg.com/00/s/NjAwWDgwMA==/z/cIk...   
3  1505788420  [https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/NmU...   
4  1516085720  [https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/fcY...   

                                               title  \
0  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...   
1  Open House August 12th from 7:00pm to 9:00pm w...   
2  One, two bedroom large unit from $1650 , Hydro...   
3  1 Bedroom Renovated Apartment for Rent - 135 T...   
4  Yonge and Finch, 1+1 unit , spacious and brigh...   

                                          address      price  \
0  100 Parkway Forest Drive, Toronto, ON, M2J 1L6  $2,209.00   
1          70 Spadina  Road, Toronto, ON, M5R 2T6  $1,587.00   
2   40 Tuxedo Ct, Scarborough, ON M1G 3S7, Canada  $1,650.00   
3        135 Tyndall Avenue, Toronto, ON, M6K 2G4  $1,949.00   
4                                 M2N 5R6, Canada  $2,100.00   

        post_published_date rental_type                 bedroom_no  \
0  2020-08-13T22:34:37.000Z   Apartment                Bedrooms: 2   
1  2020-08-13T14:02:28.000Z   Apartment  Bedrooms: Bachelor/Studio   
2  2020-07-17T14:08:19.000Z   Apartment                Bedrooms: 1   
3  2020-08-13T19:06:36.000Z   Apartment                Bedrooms: 1   
4  2020-08-06T02:49:14.000Z       Condo          Bedrooms: 1 + Den   

    bathroom_no                      utilities_list  ...       Move-In Date  \
0  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...    August 13, 2020   
1  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...  September 1, 2020   
2  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...      July 24, 2020   
3  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...    August 13, 2020   
4  Bathrooms: 1  [No: Hydro, Yes: Heat, Yes: Water]  ...     August 4, 2020   

  Pet Friendly    Size (sqft) Furnished Air Conditioning Smoking Permitted  \
0          Yes            940        No               No               Yes   
1          Yes  Not Available        No               No     Outdoors only   
2      Limited            750        No               No     Outdoors only   
3          Yes            626        No               No                No   
4           No            675        No              Yes                No   

  Barrier-free Entrances and Ramps Visual Aids Accessible Washrooms in Suite  \
0                              NaN         NaN                           NaN   
1                              NaN         NaN                           NaN   
2                              NaN         NaN                           NaN   
3                              NaN         NaN                           NaN   
4                              Yes          No                           Yes   

                            geotag  
0        (43.7733049, -79.3418471)  
1        (43.6704926, -79.4056939)  
2  (43.78136869999999, -79.230797)  
3        (43.6377125, -79.4283233)  
4        (43.7758397, -79.4142973)  

[5 rows x 28 columns]

### Adding Postal codes

In [13]:
postal_regex = re.compile(r'[M]\d[A-Z] *\d[A-Z]\d')
DF["postal_code"]= DF["address"].map(lambda x : postal_regex.search(x).group() if str(type(postal_regex.search(x)))!="<class 'NoneType'>" else None)

In [14]:
postal_code_list=[]
postal_regex = re.compile(r'[M]\d[A-Z] *\d[A-Z]\d')

for address in DF["address"]:
    try:
        postal_code_list.append(postal_regex.search(address).group())
    except AttributeError as e:
        print(address)
        print(e)
        print(type(postal_regex.search(address)))
        print("---------------------------------")

Isabella St, Toronto, ON, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
Lakeshore Blvd West/ Kipling Ave, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
Lakeshore Blvd West/ Kipling Ave, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
25 Baseball Pl, Toronto, ON M4M, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
77 Shuter St, Toronto, ON M5B, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
st paschal crt, m2m4a5, Toronto, ON
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
Warren Rd, Toronto, ON M4V, Canada
'NoneType' object has no attribute 'group'
<class 'NoneType'>
---------------------------------
Dundas St E, Ontario, Canada
'NoneType' object has no attribute 

In [15]:
DF.shape

(682, 29)

### Save standalone Kijiji dataset

In [ ]:
with open('kijijilist_standalone.pkl', 'wb') as f:
    pickle.dump(DF, f)
#with open('kijijilist_standalone.pkl', 'rb') as f:
    #DF = pickle.load(f)

### Get MCI Data

In [788]:
with open('crime_data.pkl', 'rb') as f:
    crime_data = pickle.load(f)

In [789]:
crime_data.head()

ID occurrencedate reporteddate premisetype              offence      MCI  \
0   0     2019-01-01   2019-01-01  Commercial  Assault With Weapon  Assault   
1   1     2019-01-01   2019-05-06   Apartment              Assault  Assault   
2   2     2019-01-01   2019-02-01  Commercial              Assault  Assault   
3   3     2019-01-01   2019-04-27   Apartment              Assault  Assault   
4   4     2019-01-01   2019-04-26     Outside              Assault  Assault   

  Division                          Neighbourhood       Long        Lat  
0      D55                 Greenwood-Coxwell (65) -79.320106  43.665787  
1      D43                           Woburn (137) -79.204781  43.763969  
2      D43                        West Hill (136) -79.197578  43.758972  
3      D23  Mount Olive-Silverstone-Jamestown (2) -79.592819  43.739082  
4      D11                 High Park-Swansea (87) -79.451309  43.639343

### Connect tables Housing and MCI meaningfully!

In [871]:
def be_the_bridge(DF, crime_data, radius=3):
    
    bridge = pd.DataFrame()
    for row in tqdm(DF.iterrows()):
        try:
            
            loc = row[1]['geotag']
            result=crime_data[['Lat', 'Long']].apply(lambda x: (harvesine_distance(loc, (x['Lat'], x['Long'])) if harvesine_distance(loc, (x['Lat'], x['Long'])) <=radius else None), axis=1)
            result=pd.DataFrame(result[result.notnull()], columns=['distance'])
            closest_crimes=pd.concat([crime_data.loc[result.index, 'ID'], result], axis=1)
            closest_crimes['rental_id'] = row[1]['ad_id']
            closest_crimes.rename({'ID':'crime_id'})

            bridge=pd.concat([bridge, closest_crimes])
        
        except Exception as e:
            print(e)
            
    return bridge

In [873]:
#bridge = be_the_bridge(DF, crime_data, radius=3)

In [792]:
bridge.head()

ID  distance   rental_id
53  53  1.849212  1515948914
62  62  2.996900  1515948914
80  80  1.183781  1515948914
81  81  0.989350  1515948914
94  94  1.264391  1515948914

In [22]:
#with open('bridge_table_kijiji.pkl', 'wb') as f:
#     pickle.dump(bridge, f)
with open('bridge_table_kijiji.pkl', 'rb') as f:
     bridge_table = pickle.load(f)

In [23]:
bridge_table_df= pd.DataFrame(bridge_table)

In [24]:
bridge_table_df.head()

ID  distance   rental_id
53  53  1.849212  1515948914
62  62  2.996900  1515948914
80  80  1.183781  1515948914
81  81  0.989350  1515948914
94  94  1.264391  1515948914

In [25]:
bridge_table_df.shape

(2888965, 3)

### Cleaning crime df to get it ready for merge

In [21]:
bridge_table_df_clean=bridge_table_df.copy()

NameError: name 'bridge_table_df' is not defined

In [924]:
bridge_table_df_clean["rental_id"]=bridge_table_df_clean["rental_id"].map(lambda x: "k_"+x)

In [925]:
bridge_table_df_clean.rename(columns={"ID":"crime_id",
                                "distance":"distance_rental_crime"
                               }, inplace=True)

In [926]:
bridge_table_df_clean.reset_index(inplace=True, drop=True)
bridge_table_df_clean.reset_index(inplace=True)

In [927]:
bridge_table_df_clean.rename(columns={"index":"id"
                               }, inplace=True)

In [928]:
bridge_table_df_clean=bridge_table_df_clean[["id", "crime_id","rental_id","distance_rental_crime"]]

In [929]:
print(bridge_table_df_clean.dtypes)

id                         int64
crime_id                   int64
rental_id                 object
distance_rental_crime    float64
dtype: object


In [930]:
bridge_table_df_clean.head()

id  crime_id     rental_id  distance_rental_crime
0   0        53  k_1515948914               1.849212
1   1        62  k_1515948914               2.996900
2   2        80  k_1515948914               1.183781
3   3        81  k_1515948914               0.989350
4   4        94  k_1515948914               1.264391

In [931]:
with open('bridge_table_kijiji_final.pkl', 'wb') as f:
     pickle.dump(bridge_table_df_clean, f)
with open('bridge_table_kijiji_final.pkl', 'rd') as f:
     pickle.dump(bridge_table_df_clean, f)

Preparing Kijiji Data to merege with crageslist data

In [16]:
DF_clean = DF.copy()

In [17]:
DF_clean.head(1)

ad_id                                             images  \
0  1515948914  [https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...   

                                               title  \
0  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...   

                                          address      price  \
0  100 Parkway Forest Drive, Toronto, ON, M2J 1L6  $2,209.00   

        post_published_date rental_type   bedroom_no   bathroom_no  \
0  2020-08-13T22:34:37.000Z   Apartment  Bedrooms: 2  Bathrooms: 1   

                       utilities_list  ... Pet Friendly Size (sqft) Furnished  \
0  [No: Hydro, Yes: Heat, Yes: Water]  ...          Yes         940        No   

  Air Conditioning Smoking Permitted Barrier-free Entrances and Ramps  \
0               No               Yes                              NaN   

  Visual Aids Accessible Washrooms in Suite                     geotag  \
0         NaN                           NaN  (43.7733049, -79.3418471)   

  postal_code  
0     M2J 1L6  

[1 rows x 29 columns]

In [18]:
DF_clean[DF_clean["ad_id"]=="1515948914"]

ad_id                                             images  \
0  1515948914  [https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...   

                                               title  \
0  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...   

                                          address      price  \
0  100 Parkway Forest Drive, Toronto, ON, M2J 1L6  $2,209.00   

        post_published_date rental_type   bedroom_no   bathroom_no  \
0  2020-08-13T22:34:37.000Z   Apartment  Bedrooms: 2  Bathrooms: 1   

                       utilities_list  ... Pet Friendly Size (sqft) Furnished  \
0  [No: Hydro, Yes: Heat, Yes: Water]  ...          Yes         940        No   

  Air Conditioning Smoking Permitted Barrier-free Entrances and Ramps  \
0               No               Yes                              NaN   

  Visual Aids Accessible Washrooms in Suite                     geotag  \
0         NaN                           NaN  (43.7733049, -79.3418471)   

  postal_code  
0     M2J 1L6  

[1 rows x 29 columns]

In [19]:
#take columns common to Kijiji and Craigslis
DF_clean.columns

Index(['ad_id', 'images', 'title', 'address', 'price', 'post_published_date',
       'rental_type', 'bedroom_no', 'bathroom_no', 'utilities_list',
       'wifi_and_more', 'appliances', 'personal_outdoor_space', 'amenities',
       'description', 'url', 'Parking Included', 'Agreement Type',
       'Move-In Date', 'Pet Friendly', 'Size (sqft)', 'Furnished',
       'Air Conditioning', 'Smoking Permitted',
       'Barrier-free Entrances and Ramps', 'Visual Aids',
       'Accessible Washrooms in Suite', 'geotag', 'postal_code'],
      dtype='object')

In [20]:
#Rename columns
DF_clean=DF_clean[["ad_id", "title", "price", "Size (sqft)","images","url","address",
                  "post_published_date", "geotag", "postal_code", "rental_type",
                  "bedroom_no", "bathroom_no", "Furnished", "Pet Friendly",
                  "description"]]

In [21]:
DF_clean.rename(columns={"ad_id":"id",
                         "Size (sqft)": "sqft",
                         "images": "image",
                         "bedroom_no": "bedrooms",
                         "bathroom_no": "bathrooms",
                         "Furnished": "furnished",
                         "Pet Friendly": "pet_friendly",
                        }, inplace=True)

In [22]:
DF_clean.head(2)

id                                              title      price  \
0  1515948914  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...  $2,209.00   
1  1505801433  Open House August 12th from 7:00pm to 9:00pm w...  $1,587.00   

            sqft                                              image  \
0            940  [https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40...   
1  Not Available  [https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/jms...   

                                                 url  \
0  https://www.kijiji.ca/v-apartments-condos/city...   
1  https://www.kijiji.ca/v-apartments-condos/city...   

                                          address       post_published_date  \
0  100 Parkway Forest Drive, Toronto, ON, M2J 1L6  2020-08-13T22:34:37.000Z   
1          70 Spadina  Road, Toronto, ON, M5R 2T6  2020-08-13T14:02:28.000Z   

                      geotag postal_code rental_type  \
0  (43.7733049, -79.3418471)     M2J 1L6   Apartment   
1  (43.6704926, -79.4056939)     M5R 2T6   Apartment   

                    bedrooms     bathrooms furnished pet_friendly  \
0                Bedrooms: 2  Bathrooms: 1        No          Yes   
1  Bedrooms: Bachelor/Studio  Bathrooms: 1        No          Yes   

                                         description  
0  Contact Timbercreek Communities today! Parkway...  
1                    Ask About Our RentalIncentives!

In [23]:
#change price from string to int
DF_clean["price"]= DF_clean["price"].map(lambda x: x.replace('$','').replace(',','').split('.')[0])
#DF_clean["price"][0]=DF_clean["price"].map(lambda x: int(x))

In [24]:
#Drop rows with null geocode
DF_clean.drop(DF_clean[DF_clean["geotag"].isnull()].index, inplace=True)

In [25]:
#Split geotag into lat and long
DF_clean["lat"]= DF_clean["geotag"].map(lambda x: x[0])

In [26]:
DF_clean["long"]= DF_clean["geotag"].map(lambda x: x[0])

In [27]:
#Drop geotag column
DF_clean.drop(columns=["geotag"], inplace=True)

In [28]:
#take first img in images
DF_clean["image"]= DF_clean["image"].map(lambda x: x[0:1])
#convert list to str
DF_clean["image"]=DF_clean["image"].map(lambda x: str(x)[2:-2])

In [29]:
# Change rental_type to lowercase
DF_clean["rental_type"]= DF_clean["rental_type"].map(lambda x: x.lower())

In [30]:
#Change bedrooms and bathrooms to int
DF_clean["bedrooms"]= DF_clean["bedrooms"].map(lambda x: x.split(':')[1])
DF_clean["bathrooms"]= DF_clean["bathrooms"].map(lambda x: x.split(':')[1])
#Bedrooms changinging + Den to .5
DF_clean["bedrooms"]= DF_clean["bedrooms"].map(lambda x: x.strip().replace("Bachelor/Studio", "0"))
DF_clean["bedrooms"]= DF_clean["bedrooms"].map(lambda x: x.strip().replace(" + Den", ".5"))
DF_clean["bedrooms"]= DF_clean["bedrooms"].map(lambda x: x.strip().replace("+", ""))

In [31]:
#Bathrooms
DF_clean["bathrooms"]= DF_clean["bathrooms"].map(lambda x: x.strip().replace("1.5", "1"))

In [32]:
#Adding FSA column
#DF_clean["FSA"]= DF_clean["postal_code"].map(lambda x: x.strip().split(' ')[0])
FSA_regex = re.compile(r'[M]\d[A-Z]')
DF_clean["FSA"]= DF_clean["address"].map(lambda x : FSA_regex.search(x).group() if str(type(FSA_regex.search(x)))!="<class 'NoneType'>" else None)

In [33]:
#Adding a source column to show its form kijiji
DF_clean["source"]= "Kijiji"

In [34]:
#Adding k_ before each id
DF_clean["id"]= DF_clean["id"].map(lambda x: "k_"+x.strip())

In [35]:
DF_clean[DF_clean["price"]=="Please Contact"]="0"

In [36]:
DF_clean.columns

Index(['id', 'title', 'price', 'sqft', 'image', 'url', 'address',
       'post_published_date', 'postal_code', 'rental_type', 'bedrooms',
       'bathrooms', 'furnished', 'pet_friendly', 'description', 'lat', 'long',
       'FSA', 'source'],
      dtype='object')

In [37]:
#Replace sqft = Not Available to 0
DF_clean[DF_clean["sqft"]=="Not Available"]="0"
#Remove ,
DF_clean["sqft"]= DF_clean["sqft"].map(lambda x: x.strip().replace(',', '')) 

In [38]:
#Changing furnished and pet_friendly to bool
DF_clean["pet_friendly"]=DF_clean["pet_friendly"].map(lambda x: True if (x=="Yes") else False)
DF_clean["furnished"]=DF_clean["furnished"].map(lambda x: True if (x=="Yes") else False)

In [39]:
#droping rows with no date
DF_clean.drop(DF_clean[DF_clean["post_published_date"]=="0"].index, inplace=True)
#Changing post_publised_date to datetime
DF_clean["post_published_date"]=DF_clean["post_published_date"].map(lambda x: datetime.strptime(x.split("T")[0], "%Y-%m-%d"))
#datetime.strptime(my_string, "%Y-%m-%d")


In [46]:
#Changing datatypes
convert_dict = {
                #'post_published_date': datetime,
                'lat': float, 
                'long': float,
                'price':int,
                'sqft':int,
                'bedrooms':int,
                'bathrooms':int
               } 

In [47]:
DF_clean = DF_clean.astype(convert_dict)

In [48]:
print(DF_clean.dtypes)

id                             object
title                          object
price                           int64
sqft                            int64
image                          object
url                            object
post_published_date    datetime64[ns]
lat                           float64
long                          float64
postal_code                    object
FSA                            object
rental_type                    object
bedrooms                        int64
bathrooms                       int64
furnished                        bool
pet_friendly                     bool
description                    object
source                         object
dtype: object


In [43]:
DF_clean= DF_clean[["id","title","price","sqft","image","url"
                    ,"post_published_date","address","lat","long"
                   ,"postal_code", "FSA", "rental_type", "bedrooms",
                   "bathrooms", "furnished", "pet_friendly", "description"
                   ,"source"]]

In [49]:
DF_clean.head()

id                                              title  price  \
0  k_1515948914  Parkway Forest Towers - 2 Bedroom, 1 Bathroom ...   2209   
2  k_1502301137  One, two bedroom large unit from $1650 , Hydro...   1650   
3  k_1505788420  1 Bedroom Renovated Apartment for Rent - 135 T...   1949   
4  k_1516085720  Yonge and Finch, 1+1 unit , spacious and brigh...   2100   
5  k_1517893288  Dwtn Toronto King W/John St Lux 788SF 1B/R+Den...   2550   

   sqft                                              image  \
0   940  https://i.ebayimg.com/00/s/NzY4WDEzNjU=/z/K40A...   
2   750  https://i.ebayimg.com/00/s/NjAwWDgwMA==/z/cIkA...   
3   626  https://i.ebayimg.com/00/s/NzY4WDExNTI=/z/NmUA...   
4   675  https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/fcYA...   
5   788                                                      

                                                 url post_published_date  \
0  https://www.kijiji.ca/v-apartments-condos/city...          2020-08-13   
2  https://www.kijiji.ca/v-apartments-condos/city...          2020-07-17   
3  https://www.kijiji.ca/v-apartments-condos/city...          2020-08-13   
4  https://www.kijiji.ca/v-apartments-condos/city...          2020-08-06   
5  https://www.kijiji.ca/v-apartments-condos/city...          2020-08-14   

         lat       long postal_code  FSA rental_type  bedrooms  bathrooms  \
0  43.773305  43.773305     M2J 1L6  M2J   apartment         2          1   
2  43.781369  43.781369     M1G 3S7  M1G   apartment         1          1   
3  43.637712  43.637712     M6K 2G4  M6K   apartment         1          1   
4  43.775840  43.775840     M2N 5R6  M2N       condo         1          1   
5  43.646006  43.646006     M5V 0C4  M5V       condo         1          2   

   furnished  pet_friendly                                        description  \
0      False          True  Contact Timbercreek Communities today! Parkway...   
2      False         False  Fully renovated high rise building with one an...   
3      False          True  Please view our available units by clicking on...   
4      False         False  Amazing Tridel building. Yonge and Finch. Abou...   
5      False         False  Excellent downtown location King St West/John ...   

   source  
0  Kijiji  
2  Kijiji  
3  Kijiji  
4  Kijiji  
5  Kijiji

In [45]:
DF_clean.drop(columns=["address"], inplace=True)

In [50]:
DF_clean.to_csv("kijiji_rental_final.csv", index=False)

In [28]:
#with open('kijijilist_final.pkl', 'wb') as f:
    #pickle.dump(DF_clean, f)
with open('kijijilist_final.pkl', 'wb') as f:
    #pickle.dump(DF_clean, f)
    kijiji_renting_data = pickle.load(f)

UnsupportedOperation: read

## Merging Kijiji and Cregslist

Merging Bridge tables

In [6]:
#import bridge table
bridge_table_cregslist_df=pd.read_csv("Bridge_Rental_Crime_Craigslist.csv")

In [7]:
bridge_table_cregslist_df = bridge_table_cregslist_df.loc[:, ~bridge_table_cregslist_df.columns.str.contains('^Unnamed')]

In [8]:
bridge_table_cregslist_df.head()

id  crime_id     rental_id  distance_rental_crime
0   0         1  c_7175656775               2.949897
1   1        27  c_7175656775               2.671067
2   2        36  c_7175656775               2.810086
3   3        38  c_7175656775               2.961821
4   4        39  c_7175656775               2.961821

In [15]:
start=bridge_table_cregslist_df.shape[0]

In [13]:
with open('bridge_table_kijiji_final.pkl', 'rb') as f:
     bridge_table = pickle.load(f)
bridge_table_kijiji_df= pd.DataFrame(bridge_table)

In [16]:
end=bridge_table_kijiji_df.shape[0]+start

In [18]:
list(range(start,end))[-1]

14061919

In [19]:
bridge_table_kijiji_df["id"]=list(range(start,end))

In [20]:
bridge_table_kijiji_df.to_csv('bridge_table_kijiji_final.csv', index=False)

In [954]:
bridge_table_merge=bridge_table_cregslist_df.append(bridge_table_kijiji_df, ignore_index=True)

In [956]:
bridge_table_merge.shape

(14061920, 4)

In [958]:
bridge_table_merge.to_csv('bridge_table_rental_merged.csv', index=False)